In [1]:
import soccerdata as sd
import pandas as pd
import numpy as np
import traceback
import matplotlib.pyplot as plt
from mplsoccer import Pitch,VerticalPitch
from datetime import datetime,timedelta
import requests,json,config
import matplotlib.patches as patches
import math,sys,os,random
from threading import Thread
import time,concurrent
import numpy as np
from math import isnan
# import mysql.connector
from bs4 import BeautifulSoup

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

[03/14/24 00:53:39] INFO     Custom team name replacements loaded from                                _config.py:82
                             C:\Users\Hp\soccerdata\config\teamname_replacements.json.                             

                    INFO     No custom league dict found. You can configure additional leagues in    _config.py:204
                             C:\Users\Hp\soccerdata\config\league_dict.json.                                       

In [2]:
FOTMOB_URL = config.getFotMobUrls()
FOTMOB_LEAGUE = config.getFotMobLeagueDict()
CONFIG = config.getGeneralConfig()

In [4]:
def getFotMobSeasonFixture(league,seasons): # Example [2016,2017] or "23_24"
    # # making season list
    if "list" in str(type(seasons)):
        # "%2F".join(['20'+str(x)for x in season.split('_')])
        season_name_str = [(f'{x%2000}-{(x+1)%2000}',f'20{str(x)}%2F20{str(x+1)}')for x in seasons]
        fixture_urls = [(season,FOTMOB_URL['fixture'].format(FOTMOB_LEAGUE[league],season_str)) for season,season_str in season_name_str]
        
    else:
        season_name_str = "%2F".join(['20'+str(x)for x in seasons.split('_')])
        fixture_urls = [FOTMOB_URL['fixture'].format(FOTMOB_LEAGUE[league],season_name)]
    all_fixtures = []
    for season_nm,fixture_url in fixture_urls:
        fixture_res = requests.get(fixture_url)
        # return
        df = pd.DataFrame()
        
        if fixture_res.status_code == 200 :
            try:
                fixtures = fixture_res.json()
                for match in fixtures :
                    i = df.shape[0]
                    df.loc[i, 'id']=match['id']
                    df.loc[i, 'league']=league.split('-')[-1].strip(' ')
                    df.loc[i, 'season']=season_nm
                    df.loc[i, 'url']=match['pageUrl']
                    df.loc[i, 'home_team']=match['home']['name']
                    df.loc[i, 'away_team']=match['away']['name']
                    df.loc[i, 'date']=match['status']['utcTime']
                all_fixtures.append(df)
            except JSONDecodeError as jde :
                print("Json Error ")
                print("Data Response ",fixture_res.content.decode())
                all_fixtures.append(df)
            except Exception as e :
                print("General Error ")
                print("Exception as  : ",str(e))
                all_fixtures.append(df)
    print(f"Returning {len(all_fixtures)} DFs combination")            
    return pd.concat(all_fixtures, ignore_index=True)    
# getFotMobSeasonFixture(league,season_name)
# getFotMobSeasonFixture(league,[2017,2016])
# getFotMobSeasonFixture(league,season)

In [5]:
def pre_process_dataframe_for_schedules(df_name,df_under):
    df_under = df_under
    if df_name == "WS":
        df_under = df_under.reset_index()
        df_under['date'] = pd.to_datetime(df_under['date']).dt.tz_localize(None).dt.tz_localize('UTC').dt.date
        df_under['season'] = df_under['season'].astype(str).apply(lambda x: x[:2] + '-' + x[2:])
    elif df_name == "FB" :
        df_under = df_under.reset_index()
        df_under['season'] = df_under['season'].astype(str).apply(lambda x: x[:2] + '-' + x[2:])
        df_under['date'] = df_under['date'].astype(str)
        df_under['time'] = df_under['time'].astype(str).replace('<NA>',"00:00")
        df_under['date'] = df_under['date']+' '+df_under['time']+":00"
        df_under['date'] = pd.to_datetime(df_under['date']).dt.tz_localize(None).dt.tz_localize('UTC').dt.date
    else:
        df_under = df_under.reset_index()
        # df_under['date'] = pd.to_datetime(df_under['date']) + timedelta(hours=1)
        df_under['date'] = pd.to_datetime(df_under['date'].str.split('T').str[0])
        df_under['date'] = df_under['date'].dt.tz_localize(None).dt.tz_localize('UTC').dt.date
        
    return df_under.sort_values(['date', 'home_team', 'away_team'])
# pre_process_dataframe_for_schedules('FM',FM)

In [6]:
def update_team_alias(WS_df,FB_df,FM_df,file_path):
    try:
        WS = sorted(list(WS_df['home_team'].unique()))
        FB = sorted(list(FB_df['home_team'].unique()))
        FM = sorted(list(FM_df['home_team'].unique()))
        with open(file_path,'r') as f :
            existing_alias = json.load(f)
        team_alias = {}
        for i,team in enumerate(WS):
            if team not in team_alias.keys():
                team_alias[team] = [team]
            if team != FB[i] :
                team_alias[team].append(FB[i])
            if team != FM[i] :
                team_alias[team].append(FM[i])
            team_alias[team] = list(set(team_alias[team]))
        merged_alias = {**existing_alias,**team_alias}
        with open(file_path,'w') as f :
            f.write(json.dumps(merged_alias))
        return merged_alias
    except Exception as e:
        print("Err : update_team_alias : ",e)
    return False
# team_aliases = update_team_alias(WS,FB,FM,CONFIG['team_name_path'])

In [31]:
def combine_dataframes(WS,FB,FM,team_alias):
    WS['source'] = "WhoScored"
    FM['source'] = "FotMob"
    FB['source'] = "FBref"
    df = pd.DataFrame()
    dfs = [WS,FM,FB]
    # Find the DataFrame with the least number of rows
    loop_df = min(dfs, key=len)
    try:
        for x in WS.index:
            msg = None
            # Get the row data
            WS_row = WS.loc[x]
            # row = loop_df.loc[x]
            # print("Source is :",row['source'])
            row_home_team = WS_row['home_team']
            row_away_team = WS_row['away_team']
            row_date = WS_row['date']
    
            home_team_alias = team_alias[row_home_team]
            away_team_alias = team_alias[row_away_team]
    
            #Get DF ROWS
            # WS_row = WS[(WS['home_team'].isin(home_team_alias)) & (WS['away_team'].isin(away_team_alias)) & (WS['date'] == row_date)]
            FM_row = FM[(FM['home_team'].isin(home_team_alias)) & (FM['away_team'].isin(away_team_alias)) & (FM['date'] == row_date)]
            FB_row = FB[(FB['home_team'].isin(home_team_alias)) & (FB['away_team'].isin(away_team_alias)) & (FB['date'] == row_date)]
    
            if FB_row.empty or FM.empty or WS_row.empty:
                msg = f"Problem | Home : {row_home_team} | Away : {row_away_team} | Date : {row_date} \n "
                msg += f"DFs | WhoScored : {WS_row.empty} | FBRef : {FB_row.empty} | FotMob : {FM.empty}. \n"
                # msg += f"TIME | WhoScored : ({WS['date'][x]}) | FBRef : ({FM['date'][x]}) | FotMob : ({FB['date'][x]})."
                return msg
            else:
                # Keeping only first row if all exist
                # print(WS_row.shape,FB_row.shape,FM_row.shape)
                FB_row = FB_row.iloc[0]
                FM_row = FM_row.iloc[0]
            # avoiding matches which are not played yet
            if str(FB_row['game_id']) == 'nan' or FB_row['match_report'] is None:
                continue
                
            i = df.shape[0]
            
            df.loc[i, 'competition']=WS_row['league']
            df.loc[i, 'season']=WS_row['season']
            df.loc[i, 'stage']=WS_row['stage']
            
            df.loc[i, 'date'] = row_date
            df.loc[i, 'home_team'] = row_home_team        
            df.loc[i, 'away_team'] = row_away_team
            # print("INDEX",df)
            df.loc[i, 'score'] = FB_row['score']
            
            df.loc[i, 'whoscored_id']=WS_row['game_id']
            df.loc[i, 'whoscored_url']=WS_row['url']
            
            df.loc[i, 'fbref_id']=FB_row['game_id']
            df.loc[i, 'fbref_url']=FB_row['match_report']
    
            df.loc[i, 'fotmob_id']=FM_row['id']
            df.loc[i, 'fotmob_url']=FM_row['url']
        return df
    except Exception as e:
        print("Merge DF Exception Occured : ",str(e))
        import sys,os
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print(exc_type, fname, exc_tb.tb_lineno)
        print('-------')
        print('-------', msg)
        raise e
        
# merged_df = combine_dataframes(WS,FB,FM,team_aliases)

In [32]:
def main(league,CONFIG,lock) :
    try :
        season = [x for x in range(17,24)]
        ws = sd.WhoScored(leagues=league, seasons=season)
        fb = sd.FBref(leagues=league, seasons=season)
        
        # get all 3 dfs
        print("Fetching DataFrames ... ")
        whoscored_df = ws.read_schedule()
        fbref_df = fb.read_schedule()
        fotmob_df = getFotMobSeasonFixture(league,season)

        print("Preparing Fetched DataFrames ... ")
        # prepare DFs
        WS = pre_process_dataframe_for_schedules('WS',whoscored_df)
        FB = pre_process_dataframe_for_schedules('FB',fbref_df)
        FM = pre_process_dataframe_for_schedules('FM',fotmob_df)
    
        print("Updating Team Alias Files ... ")
        # update team alias
        with lock:
            team_aliases = update_team_alias(WS,FB,FM,CONFIG['team_name_path'])
    
        # merge dfs
        print("Merging Fetched DataFrames ... ")
        merged_df = combine_dataframes(WS,FB,FM,team_aliases)
        print("DFGHJKL",str(merged_df))
        merged_df.to_csv(rf"./combined_sch/combined_{league}.csv")
        
        # get missing data info
        print("Get missing games DataFrames ... ")
        miss_WS = WS[~WS['game_id'].isin(merged_df['whoscored_id'])]
        miss_FM = FM[~FM['id'].isin(merged_df['fotmob_id'])]
        miss_FB = FB[~FB['game_id'].isin(merged_df['fbref_id'])]
    
        miss_WS.to_csv(rf"./missing_matches/miss_WS_{league}.csv")
        miss_FB.to_csv(rf"./missing_matches/miss_FB_{league}.csv")
        miss_FM.to_csv(rf"./missing_matches/miss_FM_{league}.csv")
    except Exception as e :
        print("Main Exception Occured : ",str(e))
        import sys,os
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print(exc_type, fname, exc_tb.tb_lineno)
        raise e

In [33]:
leagues = [
    # 'Belgian_Div_A',
    # 'ENG-Premier League',
    'ESP-La Liga',
    'Eredivisie',
    # 'FRA-Ligue 1',
    'GER-Bundesliga',
    'ITA-Serie A',
    'Primeira_Liga',
    'Pro_League',
    'UCL',
    'UECL',
    'UEL'
]
main(leagues[0],CONFIG,file_lock)

[03/14/24 01:59:19] INFO     Saving cached data to C:\Users\Hp\soccerdata\data\WhoScored              _common.py:91

[03/14/24 01:59:20] INFO     Saving cached data to C:\Users\Hp\soccerdata\data\FBref                  _common.py:91

Fetching DataFrames ... 


                    INFO     Retrieving game schedule of ESP-La Liga - 1718 from the cache         whoscored.py:438

                    INFO     Retrieving game schedule of ESP-La Liga - 1819 from the cache         whoscored.py:438

...

                    INFO     Scraping game schedule for Jumamosi, Ago 26 2023                      whoscored.py:327

                    INFO     Scraping game schedule for Jumapili, Ago 27 2023                      whoscored.py:327

[03/14/24 02:00:34] INFO     Scraping game schedule for Jumatatu, Ago 28 2023                      whoscored.py:327

Returning 7 DFs combination
Preparing Fetched DataFrames ... 
Updating Team Alias Files ... 
Merging Fetched DataFrames ... 
DFGHJKL Problem | Home : Leganes | Away : Deportivo Alaves | Date : 2017-08-18 
 DFs | WhoScored : False | FBRef : True | FotMob : False. 

Main Exception Occured :  'str' object has no attribute 'to_csv'
<class 'AttributeError'> 3103475529.py 28


AttributeError: 'str' object has no attribute 'to_csv'

In [15]:
import concurrent.futures
import threading

# Define a lock for file access
file_lock = threading.Lock()

# Create a thread pool executor with 4 threads
with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
    # Submit fx1 function for each team name
    futures = [executor.submit(main, league,CONFIG,file_lock) for league in leagues]

    # Wait for all tasks to complete
    concurrent.futures.wait(futures)


[03/14/24 01:03:12] INFO     Saving cached data to C:\Users\Hp\soccerdata\data\WhoScored              _common.py:91

                    INFO     Saving cached data to C:\Users\Hp\soccerdata\data\WhoScored              _common.py:91

                    INFO     Saving cached data to C:\Users\Hp\soccerdata\data\WhoScored              _common.py:91

                    INFO     Saving cached data to C:\Users\Hp\soccerdata\data\WhoScored              _common.py:91

[03/14/24 01:03:14] INFO     Saving cached data to C:\Users\Hp\soccerdata\data\FBref                  _common.py:91

...

[03/14/24 01:29:15] INFO     Scraping game schedule for Ijumaa, Jul 28 2023                        whoscored.py:327

                    INFO     Scraping game schedule for Jumamosi, Jul 29 2023                      whoscored.py:327

[03/14/24 01:29:16] INFO     Scraping game schedule for Jumapili, Jul 30 2023                      whoscored.py:327

Main Exception Occured :  'Pro_League'
<class 'KeyError'> 2029495897.py 11


D:\Visual Analysis\analysis-notebooks\notebook-env\lib\site-packages\soccerdata\fbref.py:672: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pd.concat(schedule)


In [ ]:
league = "ENG-Premier League"
# season = [2017,2018,2019,'2020','2021',2022,2023]
season = [x for x in range(17,24)]
ws = sd.WhoScored(leagues=league, seasons=season)
fb = sd.FBref(leagues=league, seasons=season)
# whoscored_df = ws.read_schedule()
# ws.seasons,fb.seasons

In [ ]:
# ws = sd.WhoScored(leagues=league, seasons=season)
# fb = sd.FBref(leagues=league, seasons=season)
# # whoscored_df = ws.read_schedule()
# ws.seasons,fb.seasons

In [ ]:
# fbref_df = fb.read_schedule()

In [ ]:
# FM[FM['date'] < datetime.now().date()]['season'].value_counts(),FM[FM['date'] < datetime.now().date()].shape,\
# FB[FB['date'] < datetime.now().date()]['season'].value_counts(),FB[FB['date'] < datetime.now().date()].shape,\
# WS[WS['date'] < datetime.now().date()]['season'].value_counts(),WS[WS['date'] < datetime.now().date()].shape

In [ ]:
# ws = sd.WhoScored(leagues=league, seasons=season)
# fb = sd.FBref(leagues=league, seasons=season)
# whoscored_df = ws.read_schedule()
# fbref_df = fb.read_schedule()
# fotmob_df = getFotMobSeasonFixture(league,season_name)
# del ws
# del fb
# WS = pre_process_dataframe_for_schedules('WS',whoscored_df)
# FB = pre_process_dataframe_for_schedules('FB',fbref_df)
# FM = pre_process_dataframe_for_schedules('FM',fotmob_df)

## Matching DFs

In [ ]:
miss_WS = WS[~WS['game_id'].isin(merged_df['whoscored_id'])]
miss_FM = FM[~FM['id'].isin(merged_df['fotmob_id'])]
miss_FB = FB[~FB['game_id'].isin(merged_df['fbref_id'])]

In [ ]:
a=miss_FM[miss_FM['date'] < datetime.now().date()]#['season']#.value_counts(),miss_FM[miss_FM['date'] < datetime.now().date()].shape,\
b=miss_FB[miss_FB['date'] < datetime.now().date()]#['season']#.value_counts(),miss_FB[miss_FB['date'] < datetime.now().date()].shape,\
c=miss_WS[miss_WS['date'] < datetime.now().date()]#['season']#.value_counts(),miss_WS[miss_WS['date'] < datetime.now().date()].shape

In [ ]:
# merged_df.to_csv("Draft_2_merged_df_all_season_prem.csv")
# merged_df.shape

In [ ]:
x = a.iloc[0]
ht = team_aliases[x['home_team']]
at = team_aliases[x['away_team']]
s = x['season']
d = x['date']

In [ ]:
# miss_WS[(miss_WS['home_team'].isin(ht)) & (miss_WS['away_team'].isin(at)) & (miss_WS['season'] == s) & (miss_WS['date'] == d) ]
WS[(WS['home_team'].isin(ht)) & (WS['away_team'].isin(at)) & (WS['season'] == s) & (WS['date'] == d) ]
# FB[(FB['home_team'].isin(ht)) & (FB['away_team'].isin(at)) & (FB['season'] == s) & (FB['date'] == d) ]
# miss_FB[(miss_FB['home_team'].isin(ht)) & (miss_FB['away_team'].isin(at)) & (miss_FB['season'] == s) & (miss_FB['date'] == d) ]

In [ ]:
fotmob_df[(fotmob_df['home_team'].isin(ht)) & (fotmob_df['away_team'].isin(at)) & (fotmob_df['season'] == s)]
# whoscored_df[(whoscored_df['home_team'].isin(ht)) & (whoscored_df['away_team'].isin(at))]
# fbref_df[(fbref_df['home_team'].isin(ht)) & (fbref_df['away_team'].isin(at))]


In [ ]:
DF = merged_df
ht,at,s

In [ ]:
DF[(DF['home_team'].isin(ht)) & (DF['season'] == s)]

In [ ]:
DF[DF['season'] == s]